Claude Code is Anthropic's official CLI tool for interacting with Claude directly from your terminal. This cheatsheet covers the essential commands, shortcuts, and configuration options.

## Installation & Setup

```bash
# Install via npm
npm install -g @anthropic-ai/claude-code

# Launch Claude Code
claude

# First run will prompt for Anthropic API key or OAuth login
```

## Essential Keyboard Shortcuts

| Shortcut | Action |
|----------|--------|
| `Enter` | Send message |
| `Shift+Enter` | New line in input |
| `Ctrl+C` | Cancel current operation |
| `Ctrl+L` | Clear screen |
| `Esc` | Interrupt Claude / Exit input mode |
| `Up/Down` | Navigate command history |
| `Tab` | Autocomplete file paths |

## Slash Commands

Type these directly in the chat:

| Command | Description |
|---------|-------------|
| `/help` | Show help and available commands |
| `/clear` | Clear conversation history |
| `/compact` | Condense conversation to save context |
| `/config` | Open configuration settings |
| `/cost` | Show token usage and cost for session |
| `/doctor` | Diagnose installation issues |
| `/init` | Initialize Claude Code in current project |
| `/login` | Switch accounts or re-authenticate |
| `/logout` | Log out of current account |
| `/memory` | View/edit persistent memory (CLAUDE.md) |
| `/model` | Switch between available models |
| `/permissions` | Manage tool permissions |
| `/review` | Request code review |
| `/status` | Show current session status |
| `/tasks` | View running background tasks |
| `/vim` | Toggle vim keybindings |

## CLI Flags

```bash
# Start with a specific prompt
claude "explain this codebase"

# Continue last conversation
claude --continue
claude -c

# Resume specific conversation
claude --resume <conversation_id>

# Print output only (no interactive mode)
claude --print "what does main.py do?"
claude -p "what does main.py do?"

# Use specific model
claude --model claude-sonnet-4-20250514

# Pipe input to Claude
cat error.log | claude "explain this error"

# Run in headless/non-interactive mode
claude --headless "fix the bug in auth.py"

# Output JSON format
claude --output-format json -p "list files"

# Set max turns for agentic loops
claude --max-turns 10 "refactor this module"

# Allow specific tools without prompting
claude --allowedTools "Bash(npm:*),Read,Write"

# Disable specific tools
claude --disallowedTools "Bash(rm:*)"
```

## Configuration Files

### CLAUDE.md (Project Memory)

Create `CLAUDE.md` in your project root to give Claude persistent context:

```markdown
# Project: MyApp

## Tech Stack
- Python 3.11, FastAPI, PostgreSQL
- Frontend: React + TypeScript

## Conventions
- Use snake_case for Python, camelCase for JS
- All API endpoints go in `src/api/`
- Tests mirror source structure in `tests/`

## Common Commands
- `make dev` - Start development server
- `make test` - Run test suite
```

### Settings Location

```bash
# Global settings
~/.claude/settings.json

# Project-level settings
.claude/settings.json

# Project-level settings (local, not committed)
.claude/settings.local.json
```

## Permission Modes

Control how Claude asks for permission:

| Mode | Behavior |
|------|----------|
| `default` | Ask before file writes and shell commands |
| `plan` | Research only, no code changes |
| `trust-tools` | Allow all tools without asking |

```bash
# Set via flag
claude --permission-mode trust-tools

# Or in settings.json
{
  "permissions": {
    "defaultMode": "default"
  }
}
```

## MCP Servers

Model Context Protocol (MCP) servers extend Claude's capabilities:

```json
// ~/.claude/settings.json
{
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-filesystem", "/path/to/dir"]
    },
    "github": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-github"],
      "env": {
        "GITHUB_TOKEN": "your-token"
      }
    }
  }
}
```

Popular MCP servers:
- `@modelcontextprotocol/server-filesystem` - Enhanced file operations
- `@modelcontextprotocol/server-github` - GitHub API integration
- `@modelcontextprotocol/server-postgres` - Database queries
- `@modelcontextprotocol/server-brave-search` - Web search

## Hooks

Run custom scripts on Claude events:

```json
// .claude/settings.json
{
  "hooks": {
    "onFileWrite": [
      {
        "command": "prettier --write",
        "match": "*.{js,ts,json}"
      }
    ],
    "onSessionStart": [
      {
        "command": "echo 'Session started'"
      }
    ]
  }
}
```

Available hooks:
- `onFileWrite` - After file is written
- `onFileRead` - After file is read
- `onSessionStart` - When session begins
- `onSessionEnd` - When session ends
- `onPromptSubmit` - Before prompt is sent

## Skills & Plugins

Skills and plugins extend Claude Code with reusable prompts and third-party integrations.

### User Skills

Create custom slash commands by adding markdown files to `~/.claude/skills/`:

```markdown
<!-- ~/.claude/skills/commit.md -->
---
description: "Generate a git commit with conventional format"
---

Review the staged changes and create a commit:
1. Analyze `git diff --staged`
2. Write a conventional commit message (feat/fix/docs/refactor/test)
3. Keep the subject line under 50 characters
4. Add body explaining "why" not "what"
```

**Skill file structure:**
```
~/.claude/skills/
├── commit.md        → /commit
├── review-pr.md     → /review-pr
├── explain.md       → /explain
└── subfolder/
    └── deploy.md    → /subfolder:deploy
```

**Skill frontmatter options:**
```yaml
---
description: "Short description shown in /help"  # Required
args: "optional arguments description"           # Optional
---
```

### Project-Level Skills

Add skills specific to a project in `.claude/skills/`:

```markdown
<!-- .claude/skills/test.md -->
---
description: "Run project tests with coverage"
---

Run the test suite for this project:
1. Execute `npm test -- --coverage`
2. If tests fail, analyze failures and suggest fixes
3. Report coverage summary
```

### Installing Plugins

Plugins are community-contributed skill packs. Install via the registry:

```bash
# Install a plugin
claude /install-plugin @username/plugin-name

# Or add manually to settings.json
```

```json
// ~/.claude/settings.json
{
  "plugins": [
    "ralph-wiggum@claude-plugins-official",
    "frontend-design@claude-plugins-official"
  ]
}
```

### Popular Plugins

| Plugin | Description |
|--------|-------------|
| `frontend-design` | Create polished UI components |
| `ralph-wiggum` | Agentic loop for complex multi-step tasks |

### Plugin Skills Usage

Plugin skills are invoked with the plugin prefix:

```bash
# Use frontend-design plugin
/frontend-design:frontend-design

# Use ralph-wiggum plugin  
/ralph-wiggum:ralph-loop
/ralph-wiggum:cancel-ralph
/ralph-wiggum:help
```

### Skill Best Practices

1. **Be specific** - Include exact commands and expected outputs
2. **Add context** - Tell Claude what files/patterns to look for
3. **Chain actions** - Skills can include multiple steps
4. **Use variables** - Reference `$ARGS` for user input

```markdown
<!-- ~/.claude/skills/scaffold.md -->
---
description: "Scaffold a new component"
args: "<component-name>"
---

Create a new React component named $ARGS:
1. Create `src/components/$ARGS/$ARGS.tsx`
2. Create `src/components/$ARGS/$ARGS.test.tsx`
3. Create `src/components/$ARGS/index.ts` barrel export
4. Use TypeScript and follow project conventions from CLAUDE.md
```

## Pro Tips

1. **Use `/compact` liberally** - Keeps context window efficient for long sessions

2. **Create project-specific CLAUDE.md** - Claude reads this automatically and follows your conventions

3. **Pipe content directly** - `git diff | claude "review these changes"` is powerful

4. **Use `--print` for scripts** - Get clean output for automation

5. **Check `/cost` periodically** - Monitor your API usage

6. **Run `/doctor`** - When things aren't working right

7. **Use background tasks** - Claude can run long operations without blocking

8. **Trust but verify** - Always review code changes before committing

## Useful Patterns

```bash
# Quick code explanation
claude -p "explain src/auth/" | less

# Debug with error logs
tail -100 app.log | claude "what's causing these errors?"

# Code review workflow
git diff main | claude "review this PR"

# Generate commit message
git diff --staged | claude -p "write a commit message"

# Explore unfamiliar codebase
claude "give me an overview of this project's architecture"

# Fix failing tests
npm test 2>&1 | claude "fix these test failures"
```

## Resources

- [Claude Code Documentation](https://docs.anthropic.com/en/docs/claude-code)
- [GitHub Repository](https://github.com/anthropics/claude-code)
- [MCP Specification](https://modelcontextprotocol.io)